In [25]:
# !pip install polars
# !pip install --upgrade s3fs

In [26]:
import boto3
import polars as pl
import pandas as pd
import s3fs
from urllib.parse import urlparse

In [8]:
# Define the S3 bucket name and any prefix you want to use
bucket_name = 'adaproject-lil0mohammedali'

# Initialize a session using Boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

for obj in bucket.objects.filter():
    print(obj.key)


df_timeseries_en.tsv
filtered_yt_metadata.csv
filtered_yt_metadata_helper.feather.csv
youtube_comments.tsv.gz


## Filtering out the relevant videos  

In [11]:
s3_feather_path = "s3://adaproject-lil0mohammedali/filtered_yt_metadata_helper.feather.csv"

filtered_df_metadata_feather = pl.read_csv(s3_feather_path)

In [14]:
s3_comments_path = "s3://adaproject-lil0mohammedali/youtube_comments.tsv.gz"

In [17]:
boto3.client("s3").head_object(Bucket=bucket_name, Key="youtube_comments.tsv.gz")

{'ResponseMetadata': {'RequestId': 'Y5PG8YSNJPC05DCK',
  'HostId': 'ZdJAS8XHka/zpn/fEkrSeXQfGsiimeV29MJQyiwwXrwpbhnd3yQEQIcn8pgTShE/WsTJcSdZ1qYux65/bOsQsMTgqmbroaM4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZdJAS8XHka/zpn/fEkrSeXQfGsiimeV29MJQyiwwXrwpbhnd3yQEQIcn8pgTShE/WsTJcSdZ1qYux65/bOsQsMTgqmbroaM4',
   'x-amz-request-id': 'Y5PG8YSNJPC05DCK',
   'date': 'Wed, 13 Nov 2024 20:16:08 GMT',
   'last-modified': 'Tue, 05 Nov 2024 18:45:34 GMT',
   'etag': '"bb895791edd0c4a265c1aa8669741b76-4495"',
   'x-amz-server-side-encryption': 'AES256',
   'x-amz-version-id': 'zal6W7tTUOWTB7EOIi4XwATphp5cKf2x',
   'accept-ranges': 'bytes',
   'content-type': 'application/x-gzip',
   'content-length': '77219676839',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2024, 11, 5, 18, 45, 34, tzinfo=tzutc()),
 'ContentLength': 77219676839,
 'ETag': '"bb895791edd0c4a265c1aa8669741b76-4495"',
 'VersionId': 'zal6W7tTUOWTB7EOIi4XwA

In [ ]:
# Initialize S3 filesystem
fs = s3fs.S3FileSystem()

# Define S3 path and batch size
s3_path = s3_comments_path
batch_size = 8600000000/100000
total_batches = 8600000000/batch_size

s3_output_path = 's3://adaproject-lil0mohammedali/filtered_youtube_comments.tsv'  # Output path in S3

i = 0
# Open the output file on S3 for writing
with fs.open(s3_output_path, "wb") as output_file:
    # Read the file in chunks directly from S3
    with fs.open(s3_path, "rb") as f:
        for chunk in pd.read_csv(f, sep='\t', compression='gzip', chunksize=batch_size):
            # Convert chunk to Polars DataFrame
            batch = pl.DataFrame(chunk)
            # Filter data based on video_id
            filtered_batch = batch.filter(pl.col("video_id").is_in(filtered_df_metadata_feather["display_id"])) 
            
            # For the first batch, include the header
            if i == 0:
                filtered_batch.write_csv(output_file, include_header=True)
            else:
                # For subsequent batches, append without the header
                filtered_batch.write_csv(output_file, include_header=False)
            
            i += 1
            print(f"batch {i} / {total_batches} \r", end='')

In [15]:
# load yt metadata in chunks and filter for videos contained in filtered_df_ch
# df.filter(pl.col("categories") == "News & Politics") to filter for categories of videos instead
batch_size = 100000
reader = pl.read_csv_batched(
    s3_comments_path,
    separator="\t",
    batch_size= batch_size
)

total_batches = 8600000000/batch_size

batches = reader.next_batches(5)
i = 0
while batches:
    for df in batches:
        if i == 0:
            df.filter(pl.col("video_id").is_in(filtered_df_metadata_feather["display_id"])).write_csv("../Youniverse/filtered_youtube_comments.tsv", include_header=True)
            #df.filter(pl.col("video_id") == "BFlU3b68aNs").write_csv("../Youniverse/filtered_youtube_comments.tsv", include_header=True)
        else:
            with open("../Youniverse/filtered_youtube_comments.tsv", "a") as fh:
                fh.write(df.filter(pl.col("video_id") == "fXN0ABkfZ7M").write_csv(file=None, include_header=False))
        i = i+1
        print(f"batch {i} / {total_batches} \r", end='')
    batches = reader.next_batches(5)

RuntimeError: BindingsError: "Os { code: 2, kind: NotFound, message: \"No such file or directory\" }"

## Checking if commment replies count as comments

Using a video for example: Search through all the youtube_comments in youtube_comments file. Get the rows corresponding to the video. Sum up total number of rows (# comments) and total number of replies. If replies > number of rows, then replies dont count as comments


In [4]:
unique_video_id: "BFlU3b68aNs"
